In [3]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [4]:
import tensorflow as tf
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

## Iris dataset

- turn it into 32bit float
- split in train and test dataset
- one hot encode target (y)
- create a tf.dataset for train and test

In [3]:
iris = datasets.load_iris()

NameError: name 'datasets' is not defined

In [4]:
# convert to float32
data = iris.data
data = data.astype(np.float32)
labels = iris.target
classes_nb = np.max(labels)+1  #3

NameError: name 'iris' is not defined

In [5]:
iris.target, iris.target.shape

NameError: name 'iris' is not defined

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)

# split in train and test
y_oh_train = tf.one_hot(y_train, len(np.unique(iris.target)), dtype=tf.float32)

train_nb = X_train.shape[0]
test_nb = X_test.shape[0]

In [0]:
# create train and test datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test))

In [0]:
# iterate over 1 sample to understand its content
# it return a tuple of tensors
next(iter(train_ds))

# Model

We create a pure tensorfow model (without using any keras facilities like layers). We only use losses, initializers from Keras.

We setup : 
- layer structure
- a loss function
- a train_on_mini_batch function
- a training loop of multiple epochs on many batches


In [0]:
# create train and test datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test))


tf.random.set_seed(42)
def get_glorot_uniform_tensor(shape):
    return tf.keras.initializers.GlorotUniform(seed=42)(shape)

def get_normal_tensor(shape):
    return tf.random.normal(shape,mean=0.0, stddev=0.05)

def get_zero_tensor(shape):
    return tf.zeros(shape)

get_glorot_uniform_tensor((3,2)).numpy(), get_normal_tensor((3,2)).numpy(), get_zero_tensor((3,2)).numpy()

![Model proposé](https://www.tensorflow.org/images/custom_estimators/full_network.png)

In [0]:
class Model(object):
    def __init__(self):
        w_init = get_glorot_uniform_tensor # how we initialize our weights        
        b_init = get_zero_tensor # bias init.

        self.W1 = tf.Variable(w_init((4,10)), dtype=tf.float32)
        self.b1 = tf.Variable(b_init((10)), dtype=tf.float32)

        self.W2 = tf.Variable(w_init((10,10)), dtype=tf.float32)
        self.b2 = tf.Variable(b_init((10)), dtype=tf.float32)

        self.W3 = tf.Variable(w_init((10,3)), dtype=tf.float32)
        self.b3 = tf.Variable(b_init((3)), dtype=tf.float32)

    def forward(self, x):

        Z1 = tf.add(tf.matmul(x, self.W1), self.b1)
        A1 = tf.nn.relu(Z1)

        Z2 = tf.add(tf.matmul(A1, self.W2), self.b2)
        A2 = tf.nn.relu(Z2)

        Z3 = tf.add(tf.matmul(A2, self.W3), self.b3)
        A3 = tf.nn.softmax(Z3)

        return A3

model = Model()
pred_train = model.forward(tf.constant(X_train,dtype=tf.float32))
#tf.reduce_sum(pred_train,axis=1)           

In [0]:
def compute_loss(y_pred,y_true):
    cce = tf.keras.losses.SparseCategoricalCrossentropy()
    loss= cce(y_true, y_pred)
    return loss

compute_loss(pred_train, y_train)


In [0]:
# Home made optimizer using tensorflow automatic computation of gradients
def train_minibatch(model, X, y_true, learning_rate=0.01):
    with tf.GradientTape() as t:
        current_loss =  compute_loss(model.forward(X), y_true)
    
    #print(model.b1)
    dW1, db1, dW2, db2, dW3, db3= t.gradient(current_loss, [model.W1, model.b1, model.W2, model.b2, model.W3, model.b3])
    #print(db1)
    model.W1.assign_sub(learning_rate * dW1)
    model.b1.assign_sub(learning_rate * db1)
    #print(model.b1)
    model.W2.assign_sub(learning_rate * dW2)
    model.b2.assign_sub(learning_rate * db2)
    model.W3.assign_sub(learning_rate * dW3)
    model.b3.assign_sub(learning_rate * db3)
    return current_loss

train_minibatch(model,
                next(iter(train_ds.batch(10)))[0], #X
                next(iter(train_ds.batch(10)))[1], #y_true
                0.5)

In [0]:
# https://datascience.stackexchange.com/questions/36450/what-is-the-difference-between-gradient-descent-and-stochastic-gradient-descent

# use keras optimizer to enhance gradient descent performances (finds optimal ways to update parameters)

def train_minibatch_with_k_optim(model,X,y_true,learning_rate=0.01):

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate) # the SGD optimizer!
    with tf.GradientTape() as t:
        current_loss =  compute_loss(model.forward(X), y_true)

    # compute gradients
    dW1, db1, dW2,db2,dW3, db3 = t.gradient(current_loss, [model.W1, model.b1,model.W2, model.b2,model.W3,model.b3])

    # apply gradient descent using SGD optimizer
    optimizer.apply_gradients(zip([dW1, db1, dW2,db2,dW3, db3], [model.W1, model.b1,model.W2, model.b2,model.W3,model.b3]))

    return current_loss
    

In [0]:
# Collect the history of W-values and b-values to plot later
W1s, b1s,W2s,b2s,W3s,b3s = [], [],[],[],[],[]
losses =[]
accuracies = []
epochs = range(100)
batch_size = train_nb // 4
cat_acc = tf.keras.metrics.SparseCategoricalAccuracy()

learning_rate = 0.1

model = Model()

for epoch in epochs:

    W1s.append(model.W1.numpy())
    b1s.append(model.b1.numpy())
    W2s.append(model.W2.numpy())
    b2s.append(model.b2.numpy())
    W3s.append(model.W3.numpy())
    b3s.append(model.b3.numpy())

    # shuffle batch into minibatches using N=train_nb random samples
    train_batch_ds = train_ds.shuffle(buffer_size=train_nb).batch(batch_size, drop_remainder=True) # use the tensorslicedataset's batch function
    batch_nb = 0

    # for each minibatch, compute and apply gradient descent 
    for x, y in train_batch_ds:
        current_loss = train_minibatch(model, x, y, learning_rate=learning_rate) # current loss
        batch_nb = batch_nb+1 # new batch

    # store epoch's actual loss
    losses.append(current_loss.numpy())

    # create a new random batch of N=train_nb samples
    X_train, y_train = next(iter(train_ds.batch(train_nb)))

    # compute ouput label probabilities at epoch
    y_pred_train = model.forward(X_train)

    # compute epoch accuracy
    acc = cat_acc(y_train,y_pred_train)
    accuracies.append(acc.numpy())
    if epoch % 10 == 0:
        print(current_loss.numpy(),acc.numpy())

## Plot learning

In [0]:
plt.plot(accuracies)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(losses)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Test model

In [0]:
X_test, y_test = next(iter(test_ds.batch(test_nb)))
y_pred_test = model.forward(X_test)
acc = cat_acc(y_test,y_pred_test)
print("accuracy of test: ",acc.numpy())

## Pure Keras 

In [0]:
k_model = tf.keras.Sequential([
  tf.keras.layers.Dense(10,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42), activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42), activation=tf.nn.relu),
  tf.keras.layers.Dense(3,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42), activation=tf.nn.softmax  )
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
k_model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
history = k_model.fit(train_ds.shuffle(buffer_size=train_nb).batch(batch_size=batch_size), epochs=20,verbose=1)
#history.history['accuracy'][-1], history.history['loss'][-1]

In [0]:
X_test, y_test = next(iter(test_ds.batch(test_nb)))
y_k_pred_test = k_model.predict(X_test)
acc = cat_acc(y_test,y_k_pred_test)
print("accuracy of test: ",acc.numpy())

### Plot keras learning process

In [0]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
tf.config.list_physical_devices()